In [46]:
#Step 1. 필요한 모듈과 라이브러리를 import 합니다

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys

import math
import numpy  
import pandas as pd  
import xlwt

#Step 2. 필요한 정보를 입력 받습니다
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
must_included = input('2. 포함할 키워드를 입력하세요 : ')
not_included = input('3. 제외할 키워드를 입력하세요 : ')

cnt = int(input('4.크롤링 할 건수는 몇건입니까?: '))

start_date = input('5. 조회 시작할 날짜를 입력하세요(예:2017-01-01) : ')
end_date = input('6. 조회 종료할 날짜를 입력하세요(예:2017-12-31) : ')


page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수 

f_name = input('3.txt 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')
fc_name = input('4.csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')
fx_name = input('5.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
s_time = time.time( ) 

path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
driver = webdriver.Chrome(path)
driver.get("https://www.naver.com")



time.sleep(2)  #  창이 모두 열릴 때 까지 2초 기다립니다.

time.sleep(2)  # 위 페이지가 모두 열릴 때 까지 2초 기다립니다.


#Step 4. 검색창의 이름을 찾아서 검색어를 입력하고 검색을 실행합니다

element = driver.find_element_by_id("query")
element.send_keys(query_txt)
element.send_keys("\n")


#블로그 탭 선택하기 

blog_tab_select = driver.find_element_by_link_text('블로그').click()

1.크롤링할 키워드는 무엇입니까?: 여름휴가
2. 포함할 키워드를 입력하세요 : 국내
3. 제외할 키워드를 입력하세요 : 해외, 외국
4.크롤링 할 건수는 몇건입니까?: 17
5. 조회 시작할 날짜를 입력하세요(예:2017-01-01) : 2017.01.01
6. 조회 종료할 날짜를 입력하세요(예:2017-12-31) : 2017.12.31
3.txt 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: /Users/kibeomkim/Desktop/ex72.txt
4.csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: /Users/kibeomkim/Desktop/ex72.txt
5.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: /Users/kibeomkim/Desktop/ex72.xlsx


In [47]:
#step 5. '여름휴가' 검색 후 상세 검색 옵션 설정하기 


try : 
    element_period = driver.find_element_by_link_text("기간")
except : 
    element_start = driver.find_element_by_class_name("bt_option")
    element_start.click()
    element_period = driver.find_element_by_link_text("기간")
    element_period.click()
else : 
    element_period.click()


element_date_from = driver.find_element_by_id("date_from_input")
element_date_from.send_keys(start_date)

element_date_to = driver.find_element_by_id("date_to_input")
element_date_to.send_keys(end_date)

element_selection = driver.find_element_by_class_name("tx")
element_selection.click()


element_sss = driver.find_element_by_xpath("/html/body/div[3]/div[1]/div[2]/div/ul/li[7]/a")
time.sleep(5) #위 xpath 찾을때 까지 5초 기다려라
element_sss.click()
time.sleep(5) #상세검색 창 띄우기까지 5초 기다려라


element_must_include = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div/ul/li[7]/div/div/div[2]/dl[2]/dd/div[2]/input')

element_must_include.send_keys(must_included)



element_must_exclude = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div/ul/li[7]/div/div/div[2]/dl[2]/dd/div[3]/input')

element_must_exclude.send_keys(not_included)


element_detail_search = driver.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div/ul/li[7]/div/div/div[3]/a[2]')
element_detail_search.click()


In [48]:
#Step 6. 페이지 번호를 바꾸면서 실제 데이터 추출하기

no = 1            # 게시글 번호 카운트용 변수
no2 =[]          # 게시글 번호 저장용 리스트
title2 = []
contents2=[]     # 게시글 내용 저장용 리스트
introduction2=[]
url2=[]         


html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
content_list = soup.find('ul','lst_total').find_all('div', 'total_wrap api_ani_send')

real_number = len(content_list)

if cnt > real_number : 
    print('입력하신 건수는 %s 건 인데, 조회된 실제 건수는 %s 라서, 최대 %s건 만큼만 출력하겠습니다.' %(cnt, real_number, real_number))


import sys
original = sys.stdout
f = open(f_name, 'a', encoding='UTF-8')
sys.stdout = f    

for i in content_list:
    # 게시글 번호 입력
    no2.append(no)
    print('1. 번호 :',no)
    
    # 게시글의 제목추출
    title = i.find('a', 'api_txt_lines total_tit').get_text() #무조건 태그 형태로 만들어야 find함수 가능하다
    title2.append(title)
    print('2. 제목 :',title.strip())

    #게시글 내용 추출 
    contents = i.find('div', 'api_txt_lines dsc_txt').get_text()
    contents2.append(contents)
    print('3. 내용 :',contents.strip())

    
    #게시글 소개 
    introduction = i.find('a', 'sub_txt sub_name').get_text()
    introduction2.append(introduction)
    print('4. 소개 : ',introduction.strip())

    
    #블로그 주소 
    url_source = i.find('a', 'api_txt_lines total_tit')['href']
    url2.append(url_source)
    print('5. 블로그 주소 :',url_source.strip())
    print('\n')
    


    no += 1
    if no > cnt : 
        break
    time.sleep(2)        # 페이지 변경 전 2초 대기 

        

f.close()
sys.stdout = original

In [49]:
#Step 7. 출력 결과를 표(데이터 프레임) 형태로 만들기
korea = pd.DataFrame()
korea['번호']=no2
korea['제목']=title2
korea['내용']=contents2
korea['블로그소개']=introduction2
korea['블로그주소']=url2

# 추출 결과를 csv 형태로 저장하기
korea.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 추출 결과를 엑셀 형태로 저장하기
korea.to_excel(fx_name , index=False)


# 추출 결과를 txt 파일로 저장하기
f = open('/Users/kibeomkim/Desktop/test11.txt', 'a',encoding='UTF-8')
f.write(str(no2))
f.write(str(title2))
f.write(str(contents2))
f.write(str(introduction2))
f.write(str(url2))

f.close( )
e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no-1))
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %f_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )